In [1]:
import requests
from datetime import datetime
from sklearn.cluster import KMeans

import json
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import json

import chardet
import re
import pandas as pd
import numpy as np
from pprint import pprint
import requests
from bixin import predict
import snownlp
import dm_pb2 as Danmaku
import matplotlib.pyplot as plt
import sklearn

In [2]:
# 1.根据bvid请求得到cid
bvid = "BV1bc411H7Gv"
def get_cid(bvid):
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=' + bvid
    res = requests.get(url).text
    json_dict = json.loads(res)
    pprint(json_dict)
    return (json_dict["data"][0]["duration"], json_dict["data"][0]["cid"], json_dict["data"][0]["part"])
(max_time, cid, part) = get_cid(bvid)
max_time = max_time * 1000

print(cid)



{'code': 0,
 'data': [{'cid': 1105215233,
           'dimension': {'height': 1920, 'rotate': 0, 'width': 3840},
           'duration': 87,
           'first_frame': 'http://i0.hdslb.com/bfs/storyff/n230422a219gylliozq0r520l6kdz8f1_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '【4K修复+60帧】小鬼 泰裤辣（原版）',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
1105215233


In [3]:
# 更新：新版api需要使用proto获取弹幕文件
def get_prot_dm(cid):
    url = 'https://api.bilibili.com/x/v2/dm/web/seg.so'
    params = {
        'type': 1,         # 弹幕类型
        'oid': cid,    # cid
        'segment_index': 1 # 弹幕分段
    }
    resp = requests.get(url, params)
    data = resp.content

    danmaku_seg = Danmaku.DmSegMobileReply()
    danmaku_seg.ParseFromString(data)

    return danmaku_seg.elems

danmu_proto = get_prot_dm(cid)

In [4]:
#得到弹幕 array
def to_list(prot):
    l = []
    for i in range(len(prot)):
        l.append({})
        l[i]["id"] = prot[i].id
        l[i]["progress"] = prot[i].progress
        l[i]["content"] = prot[i].content

    return l
listed_danmu = to_list(danmu_proto)
df = pd.DataFrame(listed_danmu)
mask = df['content'].str.len() > 10
df = df[~mask]
df['content'] = df['content'].str.replace('[^\w\s]', '')
mask = df['content'].str.len() < 1
df = df[~mask]
df.dropna(subset=['content'], inplace=True)
mask = df['content'].str.len() == 0
df = df[~mask]
print(df)

                       id  progress    content
0     1301402202560179456     23493         哥哥
1     1301445531851347456       324       我40帧
4     1301467215505580032     52312       油光满面
5     1301468690323432448     76163       音乐骤停
8     1301487277205294592     51629       脚趾扣地
...                   ...       ...        ...
3675  1312821270639102720     78124        泰酷拉
3676  1312842471142918144     74427        泰裤辣
3677  1312846005154539520      5672       沙暴送葬
3679  1312910903997566464     64855  那你就划拉不了w了
3681  1312912642494908416     80018        泰裤辣

[2869 rows x 3 columns]


C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\3047416955.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')


In [5]:
#divide into 6 parts according to time variation
def classify_sentiment(s):
    if(s<0.166):
        return 0
    if(s<0.333):
        return 1
    if(s<0.5):
        return 2
    if(s<0.666):
        return 3
    if(s<0.833):
        return 4
    if(s<=1):
        return 5
def get_sentiment(text):
    s = snownlp.SnowNLP(text)
    return classify_sentiment(s.sentiments)
index = 0
leftop = pd.DataFrame()

for i in range(6):
    start_time = max_time * i / 6
    end_time = max_time * (i+1) / 6
    tmp = df[(df['progress'] >= start_time) & (df['progress'] <= end_time)]
    tmp = tmp['content'].value_counts()
    tmp = pd.DataFrame({'content': tmp.index, 'count': tmp.values})
    mask = tmp['count'].values <= 3
    tmp = tmp[~mask]
    tmp['emo'] = tmp['content'].apply(get_sentiment)
    tmp['label'] = index
    print(tmp)
    leftop = pd.concat([leftop, tmp], axis=0)

    index+=1

leftop.to_csv("video.csv",index=False)
counts = df['content'].value_counts()
leftop
emo = leftop['emo'].value_counts()
emo
emo_df = pd.DataFrame(emo, index = emo.index)
emo_df['label'] = emo.index

emo_df.to_csv("emo.csv", index=False)
emo_df

       content  count  emo  label
0         我测深圳     33    2      0
1       给我点鸡叫声     13    1      0
2   什么都修复只会害了你     12    1      0
3         我是丁真     12    2      0
4        我死在深圳     10    1      0
5    什么都修只会害了你      9    0      0
6         我40帧      8    3      0
7          泰裤辣      7    3      0
8       给我点滋尿声      5    2      0
9         我是深圳      5    2      0
10        油光满面      4    3      0
11         抬裤拉      4    2      0
     content  count  emo  label
0        董明珠     14    2      1
1   如约而至地出现了     10    3      1
2        泰裤辣     10    3      1
3       油光满面      9    3      1
4         病句      9    2      1
5          油      8    2      1
6        泰裤啦      6    3      1
7   如约而至的出现了      6    3      1
8         好油      4    3      1
9     我还是在这里      4    3      1
10      杰夫哈迪      4    2      1
11       泰油辣      4    3      1
  content  count  emo  label
0     住医院     17    2      2
1    家电下乡     12    4      2
2     泰裤辣      8    3      2
3     yue      5    3     

,emo,label
3,27,3
2,17,2
1,5,1
4,3,4
5,2,5
0,1,0


In [6]:
# 2.根据cid请求弹幕，解析弹幕得到最终的数据

# def get_data(cid):
#     final_url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
#     final_res = requests.get(final_url)
#     final_res.encoding = chardet.detect(final_res.content)['encoding']
#     final_res = final_res.text
#     pattern = re.compile('<d.*?>(.*?)</d>')
#     match = re.compile(r'<d\s+[^>]*\bp="([^"]*)"')
#     pa = match.findall(final_res)
#
#     data = pattern.findall(final_res)
#     danmu_time = [float(item.split(',')[0]) for item in pa]
#     danmu_mode = [float(item.split(',')[1]) for item in pa]
#     danmu_size = [float(item.split(',')[2]) for item in pa]
#     danmu_color = [float(item.split(',')[3]) for item in pa]
#     danmu_abstime = [float(item.split(',')[4]) for item in pa]
#     danmu_pool = [float(item.split(',')[5]) for item in pa]
#     danmu_id = [(item.split(',')[6]) for item in pa]
#     danmu_rowid = [float(item.split(',')[7]) for item in pa]
#     print(len(data))
#     return (data, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid)
#
#
# (danmu_text, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid) = get_data(cid)

In [7]:
# # 分割弹幕为八个部分
# # 视频持续时间
# max_time = max(danmu_time)
# divided_danmu = []
# for i in range(8):
#     time_start = max_time * (i/8)
#     time_stop = max_time * ((i+1)/8)
#     for j in range(len(danmu_time)):
#         if time_start <= danmu_time[j] <= time_stop:
#             divided_danmu.append([i, danmu_text[j]])
# print(len(danmu_time))

In [8]:
# export csv
# arr = np.array(divided_danmu)
# df = pd.DataFrame(arr)
# df.to_csv("video.csv", header=["label", "text"])

In [9]:
def get_relate(bvid):
    # 设置请求参数
    params = {
        "bvid": bvid,  # 你想获取推荐视频的原始视频的bid
        "num": 1,  # 获取的推荐视频数量，最多为40
    }

    # 发送请求
    response = requests.get("https://api.bilibili.com/x/web-interface/archive/related", params=params)

    # 解析响应
    if response.status_code == 200:
        data = response.json()
        recommend_list = data.get("data")
        # 处理推荐视频列表数据
        return recommend_list
    else:
        print("请求失败")

relate_list = get_relate(bvid)
print(relate_list[0])

{'aid': 868021036, 'videos': 1, 'tid': 138, 'tname': '搞笑', 'copyright': 1, 'pic': 'http://i1.hdslb.com/bfs/archive/ea871b5bb1f9dcffe2dc7b8e279c4457c621737b.jpg', 'title': '保  护  砂  隐  村', 'pubdate': 1682670599, 'ctime': 1682670599, 'desc': '配音 ： 老D', 'state': 0, 'duration': 81, 'mission_id': 1458560, 'rights': {'bp': 0, 'elec': 0, 'download': 0, 'movie': 0, 'pay': 0, 'hd5': 0, 'no_reprint': 1, 'autoplay': 1, 'ugc_pay': 0, 'is_cooperation': 0, 'ugc_pay_preview': 0, 'no_background': 0, 'arc_pay': 0, 'pay_free_watch': 0}, 'owner': {'mid': 308693140, 'name': '声物课', 'face': 'https://i1.hdslb.com/bfs/face/6b49f7d46e78d996cd5d3895048c8c1d9302e51b.jpg'}, 'stat': {'aid': 868021036, 'view': 8470401, 'danmaku': 5984, 'reply': 6402, 'favorite': 91840, 'coin': 90413, 'share': 127317, 'now_rank': 0, 'his_rank': 11, 'like': 519219, 'dislike': 0, 'vt': 0, 'vv': 8470401}, 'dynamic': '', 'cid': 1111833604, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'short_link_v2': 'https://b23.tv/BV11V4

In [10]:
#迭代获取其他视频的推荐
index = 0
whole_related_df = pd.DataFrame()
for it in relate_list:
    danmu_relate = get_prot_dm(it['cid'])
    #preprocess
    listed_danmu = to_list(danmu_relate)
    df = pd.DataFrame(listed_danmu)
    mask = df['content'].str.len() > 10
    df = df[~mask]
    df['content'] = df['content'].str.replace('[^\w\s]', '')
    df['content'] = df['content'].str.replace('哈', '')
    df['content'] = df['content'].str.replace('嘿', '')
    df['content'] = df['content'].str.replace('啊', '')
    df['content'] = df['content'].str.replace('6', '')
    mask = df['content'].str.len() <= 1
    df = df[~mask]
    df.dropna(subset=['content'], inplace=True)
    #end of preprocess
    relate_tmp = df['content'].value_counts()
    relate_tmp = pd.DataFrame({'content': relate_tmp.index, 'count': relate_tmp.values})
    relate_tmp['emo'] = relate_tmp['content'].apply(get_sentiment)
    relate_tmp.to_csv("relate_video_"+str(index)+".csv")
    whole_related_df = pd.concat([df, whole_related_df], axis=0)

    index+=1

whole_related_df['id'] /= 100000000
whole_related_df

C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\2503830649.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\2503830649.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\2503830649.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\2503830649.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\2503830649.py:11: FutureWarning: The default value of regex will change

,id,progress,content
0,1.305140e+10,745,来了
1,1.305140e+10,163,当你看到我的时候
2,1.305140e+10,4624,芜湖
3,1.305141e+10,13096,来了
4,1.305141e+10,339,见小渔人乃大惊
...,...,...,...
3835,1.312821e+10,25448,指木叶下忍
3837,1.312841e+10,73869,这些女的
3838,1.312857e+10,64346,忍不住喇
3839,1.312900e+10,77502,太酷啦


In [11]:
# whole_count = whole_related_df['content'].value_counts()
# whole_count = pd.DataFrame({'content': whole_count.index, 'count': whole_count.values})
filtered_whole = whole_related_df[whole_related_df['progress'] > 20000]
count = len(filtered_whole)
li = [len(whole_related_df), count]
li = pd.DataFrame(li)
li = li.T
li.columns=['sum', 'recent']
li.to_csv('datasum.csv', index=False)
whole_count = whole_related_df.groupby('content').agg({'id': 'sum', 'content': 'size'}).rename(columns={'content': 'count'}).reset_index()
mask = whole_count['count'].values <= 20
whole_count = whole_count[~mask]
whole_count
whole_count['emo'] = whole_count['content'].apply(get_sentiment)

In [12]:
whole_count.to_csv("main.csv")
print(whole_count)
whole_count

max_count = whole_count['count'].max()
max_content = whole_count.loc[whole_count['count'] == max_count, 'content'].values[0]
keyword = max_content
print(keyword)

      content            id  count  emo
1275     一超多强  3.002415e+11     23    2
1786      不然呢  3.529777e+11     27    3
1998     两眼一黑  5.462750e+11     42    2
2014       严肃  3.104582e+11     27    3
2019     严肃认真  1.193895e+12    103    3
...       ...           ...    ...  ...
21272   那可太好了  6.394113e+11     49    2
21746      难绷  5.861603e+11     45    3
21801    雪豹闭嘴  5.092876e+11     39    2
22068    高下立判  3.134779e+11     24    2
22215    魑魅魍魉  8.877964e+11     68    5

[88 rows x 4 columns]
泰裤辣


In [13]:
# 降维 用于主图
# 使用t-SNE进行降维
mask = whole_count['content'].str.len() <= 1
whole_count = whole_count[~mask]
tsne = TSNE(n_components=2, random_state=2)
pca = PCA(n_components = 2)
#embedding = pca.fit_transform(whole_count.drop('content', axis=1))
embedding = tsne.fit_transform(whole_count.drop('content', axis=1))

# 创建降维后的DataFrame
df_tsne = pd.DataFrame(embedding, columns=['Dimension_1', 'Dimension_2'])
#df_tsne['Dimension_1'] /= 10000000000
whole_count.index = df_tsne.index
df_tsne['content'] = whole_count['content']
df_tsne['count'] = whole_count['count']
df_tsne['emo'] = df_tsne['content'].apply(get_sentiment)
df_tsne.to_csv("tsne.csv")
print(df_tsne)

D:\anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
D:\anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


    Dimension_1  Dimension_2 content  count  emo
0     11.246037     3.072978    一超多强     23    2
1     10.394179     0.868364     不然呢     27    3
2      9.026627    -3.797199    两眼一黑     42    2
3     11.175619     2.666007      严肃     27    3
4      6.130069    -6.831949    严肃认真    103    3
..          ...          ...     ...    ...  ...
83     8.517989    -5.017485   那可太好了     49    2
84     8.810665    -4.382471      难绷     45    3
85     9.236223    -3.187500    雪豹闭嘴     39    2
86    10.561950     2.653953    高下立判     24    2
87     7.281959    -6.674288    魑魅魍魉     68    5

[88 rows x 5 columns]


In [14]:
# 聚类
features = df_tsne[['Dimension_1', 'Dimension_2']]

# 使用K-means进行聚类
kmeans = KMeans(n_clusters=5)
labels = kmeans.fit_predict(features)

# 将标签添加到DataFrame的新列'label'中
df_tsne['label'] = labels
df_tsne

,Dimension_1,Dimension_2,content,count,emo,label
0,11.246037,3.072978,一超多强,23,2,3
1,10.394179,0.868364,不然呢,27,3,2
2,9.026627,-3.797199,两眼一黑,42,2,0
3,11.175619,2.666007,严肃,27,3,3
4,6.130069,-6.831949,严肃认真,103,3,4
...,...,...,...,...,...,...
83,8.517989,-5.017485,那可太好了,49,2,1
84,8.810665,-4.382471,难绷,45,3,1
85,9.236223,-3.187500,雪豹闭嘴,39,2,0
86,10.561950,2.653953,高下立判,24,2,2


In [15]:
#河流图
index = 0
river = pd.DataFrame()
for i in range(10):
    start_time = max_time * i / 10
    end_time = max_time * (i+1) / 10
    tmp = whole_related_df[(whole_related_df['progress'] >= start_time) & (whole_related_df['progress'] <= end_time)]
    tmp = tmp['content'].value_counts()
    tmp = pd.DataFrame({'content': tmp.index, 'count': tmp.values})
    mask = tmp['count'].values <= 3
    tmp['emo'] = tmp['content'].apply(get_sentiment)
    tmp = tmp['emo'].value_counts()
    tmp = tmp.sort_index()
    tmp = pd.DataFrame(tmp)
    tmp = tmp.T
    tmp.columns = ['emo0', 'emo1', 'emo2', 'emo3', 'emo4', 'emo5']
    river = pd.concat([river, tmp], axis=0)

In [16]:
river.insert(0, "time", [0,1,2,3,4,5,6,7,8,9])
river['time'] = [0,1,2,3,4,5,6,7,8,9]
river.to_csv("river.csv", index=False)


In [17]:
#热度图
#从推荐视频api中截取视频cid并遍历，得到的数据需要有两个label，分别对应时间和分区
# 分区的tid 动画  游戏 知识 科技 运动 生活 美食 鬼畜
tids = [1, 4, 36, 188, 234, 160, 211, 119]
def get_video_ranking(tid):
    url = "https://api.bilibili.com/x/web-interface/ranking/v2"
    params = {
        "rid": tid,
        "type": "all"
    }

    response = requests.get(url, params=params)
    data = response.json()
    cid_list = []
    if data["code"] == 0:
        video_list = data["data"]["list"]
        # 处理视频列表，进行进一步操作
        for i in video_list:
            cid_list.append(i['cid'])
    else:
        print("请求错误:", data["message"])
    return cid_list

def get_video_ranking_bid(tid):
    url = "https://api.bilibili.com/x/web-interface/ranking/v2"
    params = {
        "rid": tid,
        "type": "all"
    }

    response = requests.get(url, params=params)
    data = response.json()
    bvid_list = []
    if data["code"] == 0:
        video_list = data["data"]["list"]
        # 处理视频列表，进行进一步操作
        for i in video_list:
            bvid_list.append(i['bvid'])
    else:
        print("请求错误:", data["message"])
    return bvid_list
tid = 1
cids = get_video_ranking(tid)
print(cids)

[1119390802, 1120970737, 1121008873, 1120483706, 1122293728, 1118439668, 1120303711, 1120045422, 1120569543, 1121386715, 1120743605, 1121285578, 1121377072, 1117038860, 1119661955, 1118354754, 1118061668, 1122250444, 1119199481, 1120483034, 1118145843, 1121933876, 1117784350, 1122689468, 1117362151, 1121015357, 1120254313, 1120701620, 1115838113, 1120125929, 1119223477, 1113933657, 1121240354, 1117149357, 1120264413, 1120869541, 1120818361, 1119195788, 1121006405, 1120112614, 1112172399, 1120091918, 1116554244, 1119357128, 1121123174, 1118771614, 1120113972, 1121777976, 1122351817, 1120483010, 1120202296, 1121044389, 1119979984, 1116429796, 1114675402, 1116577832, 1119214800, 1118283345, 1120751228, 1119812665, 1119569777, 1116015751, 1114487029, 1118022217, 1122869345, 1118219400, 1120632530, 1121369690, 1121224415, 1118265727, 1121279818, 1118477139, 1120515098, 1120594164, 1119799696, 1120644424, 1115793231, 1120244467, 1120274559, 1120536015, 1119443605, 1119275399, 1122101854, 111

In [18]:
#获取各个分区中 每个视频的 包含目标字符的数量
def to_list_heat(prot): #热度图所需的获取方式
    l = []
    for i in range(len(prot)):
        l.append({})
        l[i]["id"] = prot[i].id
        l[i]["ctime"] = prot[i].ctime
        l[i]["content"] = prot[i].content
    return l
#外层循环， 分区进行
heat = pd.DataFrame()
label = 0
for tid in tids:
    cids = get_video_ranking(tid)
    #内层循环，每个分区进行搜索抓取
    for cid in cids:
        danmu_proto = get_prot_dm(cid)
        listed_danmu = to_list_heat(danmu_proto)
        if(len(listed_danmu) == 0):
            continue
        df = pd.DataFrame(listed_danmu)
        mask = df['content'].str.len() > 10
        df = df[~mask]
        df['content'] = df['content'].str.replace('[^\w\s]', '')
        mask = df['content'].str.len() < 1
        df = df[~mask]
        df.dropna(subset=['content'], inplace=True)
        mask = df['content'].str.len() == 0
        df = df[~mask]
        df['label'] = label
        heat = pd.concat([heat, df], axis=0)
    label += 1
print(heat)

C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change

                     id       ctime     content  label
0   1310232966001433856  1683283141           1      0
1   1310233127087563264  1683283160         一分钟      0
2   1310233195463545856  1683283168       好早一播放      0
3   1310233261196353280  1683283176        号养好了      0
4   1310233271757857536  1683283177         666      0
..                  ...         ...         ...    ...
32  1312492006366614784  1683552439      前排感觉要火      7
33  1312512582623884032  1683554892  哈哈哈哈哈哈哈哈哈哈      7
34  1312543970009891840  1683558634         这么快      7
35  1312549354137432832  1683559276       我去来真的      7
37  1312625551017998592  1683568359        卧龙凤雏      7

[397110 rows x 4 columns]


C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4218180664.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]', '')


In [19]:
heat

,id,ctime,content,label
0,1310232966001433856,1683283141,1,0
1,1310233127087563264,1683283160,一分钟,0
2,1310233195463545856,1683283168,好早一播放,0
3,1310233261196353280,1683283176,号养好了,0
4,1310233271757857536,1683283177,666,0
...,...,...,...,...
32,1312492006366614784,1683552439,前排感觉要火,7
33,1312512582623884032,1683554892,哈哈哈哈哈哈哈哈哈哈,7
34,1312543970009891840,1683558634,这么快,7
35,1312549354137432832,1683559276,我去来真的,7


In [20]:
# 筛选目标词汇
given_string = keyword
# 保留包含给定字符串的行
filtered_heat = heat[heat['content'].str.contains(given_string)]
filtered_heat


,id,ctime,content,label
447,1310300066351126784,1683291140,泰裤辣,0
3326,1311052881885057536,1683380882,泰裤辣,0
3491,1311090383484259072,1683385353,泰裤辣,0
3835,1311172647274637824,1683395159,泰裤辣,0
4043,1311372781610670848,1683419017,泰裤辣,0
...,...,...,...,...
254,1310328289151353856,1683294504,泰裤辣,7
255,1310645573250120448,1683332327,泰裤辣,7
995,1307885428396902144,1683003292,这件事情泰裤辣,7
83,1310962653203631872,1683370126,泰裤辣,7


In [21]:

min_time = filtered_heat['ctime'].min()
max_time = filtered_heat['ctime'].max()
print(min_time, max_time)
filtered_heat['ctime'] -= min_time

1682754828 1683603161


C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\1043624864.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_heat['ctime'] -= min_time


In [22]:
print(filtered_heat)

                       id   ctime  content  label
447   1310300066351126784  536312      泰裤辣      0
3326  1311052881885057536  626054      泰裤辣      0
3491  1311090383484259072  630525      泰裤辣      0
3835  1311172647274637824  640331      泰裤辣      0
4043  1311372781610670848  664189      泰裤辣      0
...                   ...     ...      ...    ...
254   1310328289151353856  539676      泰裤辣      7
255   1310645573250120448  577499      泰裤辣      7
995   1307885428396902144  248464  这件事情泰裤辣      7
83    1310962653203631872  615298      泰裤辣      7
69    1307241519023136256  171704   这真是泰裤辣      7

[2054 rows x 4 columns]


In [23]:
#用相同方法把ctime分成8份
max_time = filtered_heat['ctime'].max()
range_size = max_time / 6

# 创建新的"partition"列
filtered_heat['ctime'] = pd.cut(filtered_heat['ctime'], bins=[0, range_size, 2*range_size, 3*range_size, 4*range_size, 5*range_size, max_time], labels=[0, 1, 2, 3, 4, 5], include_lowest=True)

# 更新"ctime"列的值

filtered_heat

C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\737427493.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_heat['ctime'] = pd.cut(filtered_heat['ctime'], bins=[0, range_size, 2*range_size, 3*range_size, 4*range_size, 5*range_size, max_time], labels=[0, 1, 2, 3, 4, 5], include_lowest=True)


,id,ctime,content,label
447,1310300066351126784,3,泰裤辣,0
3326,1311052881885057536,4,泰裤辣,0
3491,1311090383484259072,4,泰裤辣,0
3835,1311172647274637824,4,泰裤辣,0
4043,1311372781610670848,4,泰裤辣,0
...,...,...,...,...
254,1310328289151353856,3,泰裤辣,7
255,1310645573250120448,4,泰裤辣,7
995,1307885428396902144,1,这件事情泰裤辣,7
83,1310962653203631872,4,泰裤辣,7


In [24]:
heat_counts = filtered_heat['label'].value_counts()
heat_counts

5    615
7    537
3    273
0    165
6    157
4    132
1    128
2     47
Name: label, dtype: int64

In [25]:
#制作扩散图
label_counts = filtered_heat.groupby('ctime')['label'].value_counts().reset_index(name='count')
label_counts.to_csv('heat.csv', index=False)
cur = 0
cou = 5
label_counts['pos'] = label_counts['count']

for c in label_counts.index:
    if label_counts.iloc[c][0]>cur:
        cou = 5
        cur += 1
    label_counts.at[c, 'pos'] = cou
    cou += label_counts.iloc[c][2]
label_counts.to_csv('rightbot.csv', index=False)

In [26]:
#折线图
filtered_heat['ctime'] = filtered_heat['ctime'].astype(int)
print(filtered_heat)
filtered_heat['ctime'] += 1
filtered_heat['ctime'] *= max_time/6
filtered_heat['ctime'] += min_time

# filtered_heat['ctime'] = (filtered_heat['ctime']+1) * max_time / 6 + min_time
filtered_heat['ctime'] = filtered_heat['ctime'].apply(datetime.fromtimestamp)
filtered_heat['ctime'] = filtered_heat['ctime'].apply(lambda x: x.strftime('%Y-%m-%d'))

heat_line = filtered_heat['ctime'].value_counts().reset_index().rename(columns={'index': 'ctime', 'ctime': 'count'})

print(heat_line)

                       id  ctime  content  label
447   1310300066351126784      3      泰裤辣      0
3326  1311052881885057536      4      泰裤辣      0
3491  1311090383484259072      4      泰裤辣      0
3835  1311172647274637824      4      泰裤辣      0
4043  1311372781610670848      4      泰裤辣      0
...                   ...    ...      ...    ...
254   1310328289151353856      3      泰裤辣      7
255   1310645573250120448      4      泰裤辣      7
995   1307885428396902144      1  这件事情泰裤辣      7
83    1310962653203631872      4      泰裤辣      7
69    1307241519023136256      1   这真是泰裤辣      7

[2054 rows x 4 columns]
        ctime  count
0  2023-05-07    699
1  2023-05-09    459
2  2023-05-02    288
3  2023-05-04    270
4  2023-05-06    248
5  2023-05-01     90


C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4226790809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_heat['ctime'] = filtered_heat['ctime'].astype(int)
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4226790809.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_heat['ctime'] += 1
C:\Users\djhxx\AppData\Local\Temp\ipykernel_8680\4226790809.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [27]:
#差分器
maximum = -9999
cur = heat_line.at[0, 'count']
for i in range(len(heat_line['count'])):
    delta = abs(cur - heat_line.at[i, 'count'])
    cur = heat_line.at[i, 'count']
    if delta > maximum:
        max = delta
        max_lab = i
print(max_lab)
#find the greatest video
bvids = get_video_ranking_bid(tids[max_lab+1])
print(bvids[max_lab])
(m, cid, part) = get_cid(bvids[max_lab])
print(part)
greatest = pd.DataFrame({'part':part, 'bvid':bvids[max_lab]}, index = [0])
greatest.to_csv('greatest.csv', index = False)

5
BV1dg4y1V7LG
{'code': 0,
 'data': [{'cid': 1120248231,
           'dimension': {'height': 1280, 'rotate': 0, 'width': 720},
           'duration': 168,
           'first_frame': 'http://i1.hdslb.com/bfs/storyff/n230506qnvlgonmkf55pqrxlg2tfjvu1_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '假如你女儿不小心闯进了  #冰激凌 世界怎么办？',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
假如你女儿不小心闯进了  #冰激凌 世界怎么办？


In [28]:
heat_line.sort_values(by='ctime', inplace=True)
filtered_heat
heat_line.to_csv('line.csv', index = False)

In [29]:
#关系图
# 找到内容为keyword的行
selected_rows = df_tsne[df_tsne['content'] == keyword]

# 提取对应行的标签列
selected_labels = selected_rows['label']

# 保留只包含选定标签的行
filtered_df = df_tsne[df_tsne['label'].isin(selected_labels)]

# 打印筛选后的DataFrame
print(filtered_df)

    Dimension_1  Dimension_2 content  count  emo  label
4      6.130069    -6.831949    严肃认真    103    3      4
10     4.738121    -5.751939    全体起立    208    1      4
20     5.220806    -6.477880     太酷啦    135    3      4
38     6.715495    -6.833618      救命     84    4      4
48     5.484342    -6.647285     泰裤啦    113    3      4
50     4.404709    -4.737801     泰裤辣   2302    3      4
51     4.515209    -5.073865     泰酷辣    474    3      4
63     5.231433    -6.476549    笑死我了    145    2      4
66     6.541717    -6.851439    绷不住了     84    2      4
71     5.745291    -6.736586     董明珠    103    2      4
74     4.695821    -5.561485     要来力    262    3      4
82     5.327643    -6.549423      逆天    129    1      4


In [30]:
json_data = []

# 对于每一行数据
for index, row in filtered_df.iterrows():
    # 创建字典存储每行的数据
    data = {
        "name": row['content'],
        "size": 127,
        "imports": []
    }

    # keyword，将所有其他内容添加到imports列表中
    if row['content'] == keyword:
        data['imports'] = filtered_df[filtered_df['content'] != keyword]['content'].tolist()
    else:
        data['imports'].append(keyword)

    # 将字典添加到json_data列表中
    json_data.append(data)

# 将json_data列表转换为JSON格式字符串
json_str = json.dumps(json_data, indent=4)

# 打印转换后的JSON格式字符串
print(json_str)
with open('data.json', 'w') as file:
    file.write(json_str)

[
    {
        "name": "\u4e25\u8083\u8ba4\u771f",
        "size": 127,
        "imports": [
            "\u6cf0\u88e4\u8fa3"
        ]
    },
    {
        "name": "\u5168\u4f53\u8d77\u7acb",
        "size": 127,
        "imports": [
            "\u6cf0\u88e4\u8fa3"
        ]
    },
    {
        "name": "\u592a\u9177\u5566",
        "size": 127,
        "imports": [
            "\u6cf0\u88e4\u8fa3"
        ]
    },
    {
        "name": "\u6551\u547d",
        "size": 127,
        "imports": [
            "\u6cf0\u88e4\u8fa3"
        ]
    },
    {
        "name": "\u6cf0\u88e4\u5566",
        "size": 127,
        "imports": [
            "\u6cf0\u88e4\u8fa3"
        ]
    },
    {
        "name": "\u6cf0\u88e4\u8fa3",
        "size": 127,
        "imports": [
            "\u4e25\u8083\u8ba4\u771f",
            "\u5168\u4f53\u8d77\u7acb",
            "\u592a\u9177\u5566",
            "\u6551\u547d",
            "\u6cf0\u88e4\u5566",
            "\u6cf0\u9177\u8fa3",
            "\u7

In [31]:
# # 3.保存弹幕列表
#
# def save_to_file(data):
#     with open("dan_mu.txt", mode="w", encoding="utf-8") as f:
#         for i in data:
#             f.write(i)
#             f.write("\n")
# save_to_file(data)


In [32]:
# # 1 导入相关库
# import pandas as pd
# import jieba
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# from imageio import imread

In [33]:
# # 目标视频文件的处理
# df = pd.read_csv("video.csv")
# array = np.array(df)
# content = array[:, 3]
# progress = array

In [34]:

# import warnings
# warnings.filterwarnings("ignore")
#
# # 2 读取文本文件，并使用lcut()方法进行分词
# with open("video.csv",encoding="utf-8") as f:
#     txt = f.read()
# txt = txt.split()
# data_cut = [jieba.lcut(x) for x in txt]
# data_cut
# # 3 读取停用词
# stop = [" ","道","说道","说",'了']
# # 4 去掉停用词之后的最终词
# s_data_cut = pd.Series(data_cut)
# all_words_after = s_data_cut.apply(lambda x:[i for i in x if i not in stop])
# # 5 词频统计
# all_words = []
# for i in all_words_after:
#     all_words.extend(i)
# word_count = pd.Series(all_words).value_counts()
# # 6 词云图的绘制
# # 1）读取背景图片
#
#
# # 2）设置词云参数
# wc = WordCloud(font_path="C:/Users/Windows/fonts/simhei.ttf",
#                background_color="white",
#                max_words=2000,
#                max_font_size=200,
#                random_state=42
#               )
# wc2 = wc.fit_words(word_count)
#
# # 3）绘制词云图
# plt.figure(figsize=(16,8), dpi=300)
# plt.imshow(wc2)
# plt.axis("off")
# plt.show()
# wc.to_file("ciyun.jpg")

